# Regression

Data set from Kaggle https://www.kaggle.com/datasets/sherrytp/airline-delay-analysis

## Description

### Context
The U.S. Department of Transportation's (DOT) Bureau of Transportation Statistics tracks the on-time performance of domestic flights operated by large air carriers. I came across this useful data from DOT's database at working and figured this would be a really helpful dataset: Summary information on the number of on-time, delayed, canceled, and diverted flight.

### Content
The datasets contain daily airline information covering from flight information, carrier company, to taxing-in, taxing-out time, and generalized delay reason of exactly 10 years, from 2009 to 2019. The DOT's database is renewed from 2018, so there might be a minor change in the column names.

### Schema
- |-- FL_DATE: string (nullable = true) - Flight date
- |-- OP_CARRIER: string (nullable = true) - Carrier operating the flight
- |-- OP_CARRIER_FL_NUM: integer (nullable = true) - Flight number
- |-- ORIGIN: string (nullable = true) - Origin airport
- |-- DEST: string (nullable = true) - Destination airport
- |-- CRS_DEP_TIME: integer (nullable = true) - Scheduled departure time
- |-- DEP_TIME: double (nullable = true) - Actual departure time
- |-- DEP_DELAY: double (nullable = true) - Departure delay
- |-- TAXI_OUT: double (nullable = true) - Taxi out time
- |-- WHEELS_OFF: double (nullable = true) - Wheels off time
- |-- WHEELS_ON: double (nullable = true) - Wheels on time
- |-- TAXI_IN: double (nullable = true) - Taxi in time
- |-- CRS_ARR_TIME: integer (nullable = true) - Scheduled arrival time
- |-- ARR_TIME: double (nullable = true) - Actual arrival time
- |-- ARR_DELAY: double (nullable = true) - Arrival delay
- |-- CANCELLED: double (nullable = true) - Cancelled flight
- |-- CANCELLATION_CODE: string (nullable = true) - Cancellation code
- |-- DIVERTED: double (nullable = true) - Diverted flight
- |-- CRS_ELAPSED_TIME: double (nullable = true) - Scheduled elapsed time
- |-- ACTUAL_ELAPSED_TIME: double (nullable = true) - Actual elapsed time
- |-- AIR_TIME: double (nullable = true) - Air time
- |-- DISTANCE: double (nullable = true) - Distance to destination
- |-- CARRIER_DELAY: double (nullable = true) - Carrier delay
- |-- WEATHER_DELAY: double (nullable = true) - Weather delay
- |-- NAS_DELAY: double (nullable = true) - NAS delay
- |-- SECURITY_DELAY: double (nullable = true) - Security delay
- |-- LATE_AIRCRAFT_DELAY: double (nullable = true) - Late aircraft delay
- |-- Unnamed: 27: string (nullable = true) - Unknown

### Goals
Predict the departure delay of a flight.


In [1]:
!kaggle datasets download -d sherrytp/airline-delay-analysis -f "airline delay analysis/2018.csv"
!unzip 2018.csv.zip
!rm 2018.csv.zip

 98%|███████████████████████████████████████ | 233M/238M [00:05<00:00, 46.0MB/s]
100%|████████████████████████████████████████| 238M/238M [00:05<00:00, 43.4MB/s]
Archive:  2018.csv.zip
  inflating: 2018.csv                


In [2]:
# initlize pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("airline-delay-regression").getOrCreate()

Picked up JAVA_TOOL_OPTIONS:  -Xmx3435m
Picked up JAVA_TOOL_OPTIONS:  -Xmx3435m
22/05/14 15:17:35 WARN Utils: Your hostname, fawazalesay-pysparkairl-mlxxsdnyoem resolves to a loopback address: 127.0.0.1; using 10.0.5.2 instead (on interface ceth0)
22/05/14 15:17:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/14 15:17:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/14 15:17:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/14 15:17:37 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Needed to make Jupyter work with Gitpod
import plotly.io as pio
pio.renderers.default = 'iframe_connected'

In [4]:
# Read the data into a dataframe and print the schema
df = spark.read.csv("2018.csv", header=True, inferSchema=True)
df.printSchema()

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: do

# Preprocessing

We dropped all the columns that are not needed for the analysis or that would be considered cheating.

We also added DAY, MONTH, and YEAR columns to the dataset.

In [5]:
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

df = df.drop("Unnamed: 27", "LATE_AIRCRAFT_DELAY", "SECURITY_DELAY", "NAS_DELAY", "WEATHER_DELAY", "CARRIER_DELAY", "AIR_TIME", "ACTUAL_ELAPSED_TIME", "DIVERTED", "CANCELLATION_CODE", "CANCELLED", "ARR_TIME", "TAXI_IN", "WHEELS_ON", "WHEELS_OFF", "TAXI_OUT", "DEP_TIME", "OP_CARRIER_FL_NUM", "ARR_DELAY")

df.printSchema()

@udf(returnType=IntegerType())
def get_month(date):
    return int(date.split("-")[1])

@udf(returnType=IntegerType())
def get_day(date):
    return int(date.split("-")[2])

@udf(returnType=IntegerType())
def get_year(date):
    return int(date.split("-")[0])

# Adds month and a day column to the dataframe
df = df.withColumn("YEAR", get_year(df["FL_DATE"]).cast(IntegerType()))
df = df.withColumn("MONTH", get_month(df["FL_DATE"]).cast(IntegerType()))
df = df.withColumn("DAY", get_day(df["FL_DATE"]).cast(IntegerType()))

df = df.drop("FL_DATE")

df = df.dropna()

df.printSchema()

# Print the first 5 rows of the dataframe
df.show(5)
print(df.count())

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)

root
 |-- OP_CARRIER: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)



+----------+------+----+------------+---------+------------+----------------+--------+----+-----+---+
|OP_CARRIER|ORIGIN|DEST|CRS_DEP_TIME|DEP_DELAY|CRS_ARR_TIME|CRS_ELAPSED_TIME|DISTANCE|YEAR|MONTH|DAY|
+----------+------+----+------------+---------+------------+----------------+--------+----+-----+---+
|        UA|   EWR| DEN|        1517|     -5.0|        1745|           268.0|  1605.0|2018|    1|  1|
|        UA|   LAS| SFO|        1115|     -8.0|        1254|            99.0|   414.0|2018|    1|  1|
|        UA|   SNA| DEN|        1335|     -5.0|        1649|           134.0|   846.0|2018|    1|  1|
|        UA|   RSW| ORD|        1546|      6.0|        1756|           190.0|  1120.0|2018|    1|  1|
|        UA|   ORD| ALB|         630|     20.0|         922|           112.0|   723.0|2018|    1|  1|
+----------+------+----+------------+---------+------------+----------------+--------+----+-----+---+
only showing top 5 rows



7096202


# Preparing data for training

We use StringIndexer to convert the categorical variables to numerical values.

We use OneHotEncoder to convert the numerical values (in ordinal form) to nominal values.

We use VectorAssembler to convert the numerical variables to a feature vector (required by the model).

We normalize the feature vector using MinMaxScaler.

In [7]:
# Preparing the data
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import MinMaxScaler

train, test = df.randomSplit([0.8, 0.2])

# Encode the categorical features using StringIndexer
indexer = StringIndexer(inputCols=["OP_CARRIER", "ORIGIN", "DEST", "DAY", "MONTH"], outputCols=["OP_CARRIER_INDEX", "ORIGIN_INDEX", "DEST_INDEX", "DAY_INDEX", "MONTH_INDEX"])

# Use one hot encoding to encode the categorical features
encoder = OneHotEncoder(inputCols=["OP_CARRIER_INDEX", "ORIGIN_INDEX", "DEST_INDEX", "DAY_INDEX", "MONTH_INDEX"], outputCols=["OP_CARRIER_VEC", "ORIGIN_VEC", "DEST_VEC", "DAY_VEC", "MONTH_VEC"])

# Create the assembler
assembler = VectorAssembler(inputCols=["OP_CARRIER_VEC", "ORIGIN_VEC", "DEST_VEC", "CRS_DEP_TIME", "CRS_ARR_TIME", "DAY_VEC", "MONTH_VEC", "YEAR"], outputCol="features")

# MinMaxScaler
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

# Get the maximum number of all categorical features in the dataframe
num_of_origins = df.select("ORIGIN").distinct().count()
num_of_destinations = df.select("DEST").distinct().count()
num_of_carriers = df.select("OP_CARRIER").distinct().count()
max_num_of_categorical_features = max(num_of_origins, num_of_destinations, num_of_carriers)
print("max categories:", max_num_of_categorical_features)

max categories: 358


# First Model: Linear Regression

In [8]:
# Linear Regression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

lr = LinearRegression(featuresCol="scaledFeatures", labelCol="DEP_DELAY", regParam=0.3)

# Create the pipeline
pipeline = Pipeline(stages=[indexer, encoder, assembler, scaler, lr])

# Train the model
model = pipeline.fit(train)

# Evaluate the model
predictions = model.transform(test)

# Print the RMSE
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="DEP_DELAY", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("RMSE:", rmse)

22/05/14 15:22:12 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/05/14 15:22:12 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/05/14 15:22:29 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


RMSE: 44.57553449188212


In [ ]:
print("Mean:", test.select("DEP_DELAY").agg({"DEP_DELAY": "mean"}).collect()[0][0])
print("Std:", test.select("DEP_DELAY").agg({"DEP_DELAY": "stddev"}).collect()[0][0])

Mean: 9.93221958518358


Std: 44.739548581249586


# Second Model: Decision Tree Regressor

In [9]:
# Decision Tree Regression Model
from pyspark.ml.regression import DecisionTreeRegressor

dtr = DecisionTreeRegressor(featuresCol="scaledFeatures", labelCol="DEP_DELAY", maxDepth=5, maxBins=max_num_of_categorical_features)

# Create the pipeline
pipeline = Pipeline(stages=[indexer, encoder, assembler, scaler, dtr])

# Train the model
model = pipeline.fit(train)

# Evaluate the model
predictions = model.transform(test)

# Print the RMSE
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="DEP_DELAY", metricName="rmse")
print("RMSE (Decision Tree Regression):", evaluator.evaluate(predictions))

22/05/14 15:30:49 WARN MemoryStore: Not enough space to cache rdd_221_2 in memory! (computed 70.7 MiB so far)
22/05/14 15:30:49 WARN BlockManager: Persisting block rdd_221_2 to disk instead.
22/05/14 15:30:49 WARN MemoryStore: Not enough space to cache rdd_221_3 in memory! (computed 46.7 MiB so far)
22/05/14 15:30:49 WARN BlockManager: Persisting block rdd_221_3 to disk instead.
22/05/14 15:30:49 WARN MemoryStore: Not enough space to cache rdd_221_4 in memory! (computed 46.7 MiB so far)
22/05/14 15:30:49 WARN BlockManager: Persisting block rdd_221_4 to disk instead.
22/05/14 15:30:51 WARN MemoryStore: Not enough space to cache rdd_221_1 in memory! (computed 109.6 MiB so far)
22/05/14 15:30:51 WARN BlockManager: Persisting block rdd_221_1 to disk instead.
22/05/14 15:30:54 WARN MemoryStore: Not enough space to cache rdd_221_5 in memory! (computed 167.4 MiB so far)
22/05/14 15:30:54 WARN BlockManager: Persisting block rdd_221_5 to disk instead.
22/05/14 15:30:54 WARN MemoryStore: Not eno

RMSE (Decision Tree Regression): 44.77400386137411
